In [1]:
import numpy as np
from pycococreatortools import pycococreatortools
import os
import glob
import cv2
from tqdm import tqdm
import json
from PIL import Image


In [2]:
home_path=os.path.expanduser('~')
dir0_with_images_and_ann = os.path.join(home_path, 'Pobrane/images/More-weapons-1')

In [3]:
def get_segmentation(in_dict):
    segmentation = []
    if not len(in_dict['contours']):
        return segmentation
    in_dict['contours'] = sorted(in_dict['contours'], key=len, reverse=True)
    for contour_pack in [in_dict['contours'][0]]:
        for contour in contour_pack:
            segmentation.append(contour['x'])
            segmentation.append(contour['y'])

    
    return [segmentation]

def get_bbox(in_dict):
    left = in_dict['bbox']['left']
    top = in_dict['bbox']['top']
    right = in_dict['bbox']['right']
    bottom = in_dict['bbox']['bottom']
    width = right-left
    height = bottom-top
    return [left, top, width, height]

In [4]:
w_id = 1
cats_ids = {"person":2,"weapon":w_id, 'People with weapons':2, 'Glock_17':w_id ,
            '45_ACP_Smith_And_Wesson ' :w_id, 'mp5k' : w_id, 'LeftHandedWeapons':w_id,
            'M39_EMR':w_id, 'M1911A':w_id, 'AKM':w_id, 'AK47': w_id, 'Colt_M4A1 ': w_id,
           'Colt_1851': w_id, 'Colt_Python':w_id, 'Remington_R1': w_id, 'AK_74': w_id, 'Beretta_R93': w_id,
            'thompson':w_id, 'Browning_HP':w_id, 'SS2_AssaultRifle':w_id, 'Scar_L': w_id, 'Dan_Wesson_715':w_id,
           'AR':w_id,'Heckler_Koch_UMP-45' : w_id, 'Remington_700_CDL': w_id, 'VKS_Vyhlop':w_id, 'AB-10_LP':w_id,
            'Enfield':w_id, 'skorpion':w_id, 'mac11':w_id}
image_id = 0
ann_id = 0
coco_output = {'images': [], "annotations": [], 'type': "instances"}
coco_output['categories'] = [{"supercategory": "person","id": cats_ids['person'],"name": "person"},
    {"supercategory": "weapon","id": cats_ids['weapon'],"name": "weapon"}]
tags = set()
for src_dir in [dir0_with_images_and_ann]:
    
    images_pathes = glob.glob(src_dir+'/*.png')
    for image_path in tqdm(images_pathes):
        image_id += 1
        image = Image.open(image_path)
        parent_dir, image_name  = image_path.split('/')[-2:]
        image_filename_with_parentdir = os.path.join(parent_dir, image_name)
        image_info = pycococreatortools.create_image_info(
        image_id, image_filename_with_parentdir, image.size)
        
        cutted_image_path = '_'.join(image_path.split('_')[:-1])
        annotation_list = glob.glob(cutted_image_path+'_*.json')
        prev_ann_id = ann_id
        for annot_path in annotation_list:
            with open(annot_path, 'r') as f:
                desc = json.load(f)
                if desc['tagName'] not in tags and desc['tagName'] not in cats_ids:
                    print(desc['tagName'])
                    tags.add(desc['tagName'])
                
                if desc['tagName'] not in cats_ids:
                    continue
                
                annotation_info = {}
                annotation_info['segmentation'] = get_segmentation(desc)
                if not len(annotation_info['segmentation']):
                    continue
                ann_id += 1
                annotation_info['area'] = desc['area']
                annotation_info['iscrowd'] = 0
                annotation_info['image_id'] = image_id
                annotation_info['bbox'] = get_bbox(desc)
                annotation_info['category_id'] = cats_ids[desc['tagName']]
                annotation_info['id'] =  ann_id
                coco_output['annotations'].append(annotation_info)
        if prev_ann_id != ann_id:
            coco_output["images"].append(image_info)

  0%|          | 1/76389 [00:01<23:40:30,  1.12s/it]

book
laptop
cell phone
table
mouse
magazine


  0%|          | 4/76389 [00:04<23:44:36,  1.12s/it]

candles


  0%|          | 5/76389 [00:05<23:39:00,  1.11s/it]

...

  0%|          | 28/76389 [00:30<23:17:14,  1.10s/it]

sedan
crane truck
jeep
bus


  0%|          | 32/76389 [00:34<22:59:51,  1.08s/it]

Rear window
van
prime mover
Tail light
Door window
Car
cabriolet


  0%|          | 45/76389 [00:48<22:55:42,  1.08s/it]


In [5]:
annotation_file='/home/m/Pobrane/trainval.json'
os.makedirs(os.path.split(annotation_file)[0], exist_ok=True)
with open(annotation_file, 'w') as f:
    json.dump(coco_output, f)